# Statistical Language Modeling with NLTK

- Evgeny A. Stepanov
- stepanov.evgeny.a@gmail.com

## Objectives

- Understanding:
    - relation between lexicon and language model
    - smoothing techniques 
    - OOV effects and remedies

- Learning how to
    - prepare data for ngram modeling
    - count ngrams in a corpus
    - train an ngram model with `NLTK`
    - use ngram model to
        - compute probability of a sequence
        - generate a sequence
    
    - evaluate ngram model
    

### Recommended Reading

- Dan Jurafsky and James H. Martin's __Speech and Language Processing__ ([3rd ed. draft](https://web.stanford.edu/~jurafsky/slp3/))

### Covered Material
- SLP
    - [Chapter 3: N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf) 

### Requirements

- [NLTK](https://www.nltk.org/)

## 1. Ngrams and Ngram Counting

[n-gram](https://en.wikipedia.org/wiki/N-gram) is a contiguous sequence of *n* items from a given sequence of text or speech. An n-gram model models sequences, notably natural languages, using the statistical properties of n-grams.

__Example__:

- character n-grams: cat
- word n-grams: the cat is fat


|                     | 1-gram  | 2-gram  | 3-gram  |
|---------------------|---------|---------|---------|
|                     | unigram | bigram  | trigram |
| *Markov Order*      | 0       | 1       | 2       |
| *Character N-grams* | `['c', 'a', 't']` | `['ca', 'at']` | `['cat']` |
| *Word N-grams*      | `['the', 'cat', 'is' , 'fat']` | `['the cat', 'cat is', ...]` | `['the cat is', ...]` |


### 1.1. Counting Ngrams

*Frequency List* of a corpus is essentially a unigram count. Ngram count only differs in a unit of counting -- sequence of $n$ of tokens. We can compute bigram count by taking sequences of 2 items, trigrams - 3, etc.

#### 1.1.1. Preparing Data
The required data format for ngram counting is a _list-of-lists_ (lists of sentences consisting of lists of words): 

```
[
     ['the', 'cat', 'is', 'fat'], 
     ['the', 'dog', 'is', 'hungry']
]
```

#### 1.1.2. Sentence beginning & end tags

Including sentence boundary markers leads to a better model. To do that we need to augment each sentence with a special symbols for beginning and end of sentence tags (`<s>` and `</s>`, respectively). The beginning of the sentence tag gives the bigram context of the first word; and encodes probability of a word to start a sentence. Adding the end of the sentence tag, on the other hand, makes the bigram model a true probability distribution (Jurafsky and Martin). "Without it, the sentence probabilities for all sentences of a given length would sum to one. This model would define an infinite set of probability distributions, with one distribution per sentence length."

For larger ngrams, we’ll need to assume extra context for the contexts to the left and right of the sentence boundaries. For example, to compute trigram probabilities at the very beginning of the sentence, we can use two pseudo-words for the first trigram (i.e. `['<s>', '<s>', w1]`). Alternatively, we can use [back-off](https://en.wikipedia.org/wiki/Katz%27s_back-off_model), and use the `['<s>', w1]` bigram probability. 

**Example**:
`['<s>', 'the', 'cat', 'is', 'fat', '</s>']`

#### 1.1.3. NLTK Utility Functions
NLTK provides utility functions for padding sequences.

In [1]:
sent = ['the', 'cat', 'is', 'fat']

In [2]:
from nltk.util import pad_sequence

list(pad_sequence(
        sent,  # input sequence
        pad_left=True,
        left_pad_symbol="<s>",
        pad_right=True,
        right_pad_symbol="</s>",
        n=2  # padding for bigrams
    )
)

['<s>', 'the', 'cat', 'is', 'fat', '</s>']

Another NLTK function wraps this utility function with default arguments to provide a more convenient interface. 

In [3]:
from nltk.lm.preprocessing import pad_both_ends

list(pad_both_ends(sent, n=2))

['<s>', 'the', 'cat', 'is', 'fat', '</s>']

#### 1.1.4. Extracting Ngrams
NLTK provides a function to extact ngrams from a sequence, which also performs padding, if required arguments are provided.

In [4]:
from nltk.util import ngrams

list(ngrams(sent, 2))

[('the', 'cat'), ('cat', 'is'), ('is', 'fat')]

In [5]:
bigrams = ngrams(
    sent,
    2,
    pad_left=True,
    left_pad_symbol="<s>",
    pad_right=True,
    right_pad_symbol="</s>",
)
list(bigrams)

[('<s>', 'the'), ('the', 'cat'), ('cat', 'is'), ('is', 'fat'), ('fat', '</s>')]

Additionally, NLTK provides wrapper functions to extract bigrams and trigrams.

In [6]:
from nltk.util import bigrams, trigrams

print(list(bigrams(sent)))
print(list(trigrams(sent)))

print(list(bigrams(pad_both_ends(sent, n=2))))
print(list(bigrams(sent, pad_left=True, left_pad_symbol="<s>", pad_right=True, right_pad_symbol="</s>")))

[('the', 'cat'), ('cat', 'is'), ('is', 'fat')]
[('the', 'cat', 'is'), ('cat', 'is', 'fat')]
[('<s>', 'the'), ('the', 'cat'), ('cat', 'is'), ('is', 'fat'), ('fat', '</s>')]
[('<s>', 'the'), ('the', 'cat'), ('cat', 'is'), ('is', 'fat'), ('fat', '</s>')]


#### 1.1.5. Extracting "Everygrams"
To make an ngram model more robust, it is usually also trained on lower-ngrams (i.e. unigrams in case of bigrams). 
NLTK also provides utility function to extract these together.

Note the `max_len` argument that defines the maximum length of an ngram.

In [7]:
from nltk.util import everygrams

print(list(everygrams(sent, max_len=2)))
print(list(everygrams(pad_both_ends(sent, n=2), max_len=2)))

[('the',), ('the', 'cat'), ('cat',), ('cat', 'is'), ('is',), ('is', 'fat'), ('fat',)]
[('<s>',), ('<s>', 'the'), ('the',), ('the', 'cat'), ('cat',), ('cat', 'is'), ('is',), ('is', 'fat'), ('fat',), ('fat', '</s>'), ('</s>',)]


In [8]:
all_ngrams = everygrams(
    sent,
    min_len=1,
    max_len=2,
    pad_left=True,
    left_pad_symbol="<s>",
    pad_right=True,
    right_pad_symbol="</s>"
)
list(all_ngrams)

[('<s>',),
 ('<s>', 'the'),
 ('the',),
 ('the', 'cat'),
 ('cat',),
 ('cat', 'is'),
 ('is',),
 ('is', 'fat'),
 ('fat',),
 ('fat', '</s>'),
 ('</s>',)]

#### 1.1.6. "Flattening" the Data
For language model training and evaluation (as well as vocabulary extraction) NLTK expects the data to be a flat list. 
In python, this is accomplished either by using `itertools.chain.from_iterable` or python list comprehension as

`[element for sublist in superlist for element in sublist]`

In [9]:
from itertools import chain

data = [['the', 'cat', 'is', 'fat'], ['the', 'dog', 'is', 'hungry']]

list(chain.from_iterable(data))

['the', 'cat', 'is', 'fat', 'the', 'dog', 'is', 'hungry']

In [10]:
[token for sent in data for token in sent]

['the', 'cat', 'is', 'fat', 'the', 'dog', 'is', 'hungry']

In [11]:
from nltk.lm.preprocessing import flatten
list(flatten(data))

['the', 'cat', 'is', 'fat', 'the', 'dog', 'is', 'hungry']

#### 1.1.7. Combining the Tasks
NLTK wraps both tasks:

- padded ngram (everygram) extraction from each sentence
- flat list creation

into a convenient utility function `padded_everygram_pipeline` that returns lazy iterators.

In [12]:
from nltk.lm.preprocessing import padded_everygram_pipeline

padded_ngrams, flat_text = padded_everygram_pipeline(2, data)

In [13]:
for sent_ngrams in padded_ngrams:
    print(list(sent_ngrams))

[('<s>',), ('<s>', 'the'), ('the',), ('the', 'cat'), ('cat',), ('cat', 'is'), ('is',), ('is', 'fat'), ('fat',), ('fat', '</s>'), ('</s>',)]
[('<s>',), ('<s>', 'the'), ('the',), ('the', 'dog'), ('dog',), ('dog', 'is'), ('is',), ('is', 'hungry'), ('hungry',), ('hungry', '</s>'), ('</s>',)]


In [14]:
list(flat_text)

['<s>',
 'the',
 'cat',
 'is',
 'fat',
 '</s>',
 '<s>',
 'the',
 'dog',
 'is',
 'hungry',
 '</s>']

### 1.2. Ngram Counter
NLTK provides NgramCounter class to do the ngram counting. The class can be initialized without any agrument and then updated with ngrams (via `update()` method); or directly initialized with ngrams.

`N()` method returns total number of ngrams stored.

In [15]:
from nltk.lm import NgramCounter

counter = NgramCounter()
counter.N()  # return total number of stored ngrams

0

In [16]:
padded_ngrams, flat_text = padded_everygram_pipeline(2, data)
counter.update(padded_ngrams)  # update counter with ngrams
counter.N()

22

In [17]:
padded_ngrams, flat_text = padded_everygram_pipeline(2, data)
counter = NgramCounter(padded_ngrams)
counter.N()

22

#### 1.2.1. Accessing Ngram Counts

Ngram counts can be accessed using standard python dictionary notation. 
- Ngram order counts can be accessed using integer keys (order)
    - returns `Frequency Distribution` or `Conditional Frequency Distribution` objects. [link](https://www.nltk.org/api/nltk.probability.html)
- Unigram counts can be accessed using string keys.
- Bigram counts can be accessed using list keys & string keys (see example)

In [18]:
# ngram order counts
print(counter[1])  # Frequency Distribution
print(counter[2])  # Conditional Frequency Distribution

<FreqDist with 8 samples and 12 outcomes>
<ConditionalFreqDist with 7 conditions>


In [19]:
# print counts
print(counter.unigrams)  # unigram count for convenience

<FreqDist with 8 samples and 12 outcomes>


In [20]:
# unigram counts
counter['the']

2

In [21]:
# 'full' bigram counts (full bigrams)
counter[['the']]['cat']

1

In [22]:
# to get a frequency distribution over all continuations, you can use list or tuple. 
# counter[['the']] or counter[('the',)]
counter[['the']]

FreqDist({'cat': 1, 'dog': 1})

In [23]:
# For Conditional Frequency Distributions, only tuples are accepted.
# counter[2][['the']] with throw an error
counter[2][('the',)]

FreqDist({'cat': 1, 'dog': 1})

### Exercise: Counting Ngrams in Shakespeare's Hamlet

- Load Shakespeare's Hamlet from Gutenberg corpus
    - lowercase it

- Extract padded unigrams and bigrams

- Using NgramCounter
    - get total number of ngrams
    - get count of unigram `the`
    - get count of bigram `of the`

In [24]:
from nltk.corpus import gutenberg

hamlet = gutenberg.sents('shakespeare-hamlet.txt')

print(len(hamlet))
print(hamlet[0])

3106
['[', 'The', 'Tragedie', 'of', 'Hamlet', 'by', 'William', 'Shakespeare', '1599', ']']


In [25]:
# lowercasing
hamlet = [[w.lower() for w in sent] for sent in hamlet]
print(hamlet[0])

['[', 'the', 'tragedie', 'of', 'hamlet', 'by', 'william', 'shakespeare', '1599', ']']


## 2. Vocabulary and Basic Usage

[`Vocabulary`](https://www.nltk.org/api/nltk.lm.vocabulary.html) class of NLTK satisfies two common language modeling requirements for a vocabulary:
- When checking membership and calculating its size, filters items by comparing their counts to a cutoff value.
- Adds a special "unknown" token which unseen words are mapped to.

Tokens with counts greater than or equal to the cut-off value will be considered part of the vocabulary.

`Vocabulary(counts=None, unk_cutoff=1, unk_label='<UNK>)` create a new `Vocabulary`.

- `counts` (optional) iterable or collections. 
    - Counter instance to pre-seed the `Vocabulary`. 
    - In case it is iterable, counts are calculated.

- `unk_cutoff` (int) - Words that occur less frequently than this value are not considered part of the vocabulary.

- `unk_label` - Label for marking words not part of vocabulary.

In [26]:
from nltk.lm import Vocabulary
hamlet_words = gutenberg.words('shakespeare-hamlet.txt')

# lowercase
hamlet_words = [w.lower() for w in hamlet_words]

# initialize vocabulary with cut-off
vocab = Vocabulary(hamlet_words, unk_cutoff=2)

`Vocabulary` methods:

- `lookup()` looks up words in a vocabulary. 
    - "Unseen" words (with counts less than cutoff) are looked up as the unknown label. 
    - If given one word (a string) as an input, this method will return a string.
    - If given a sequence, it will return an tuple of the looked up words.
    
- `update()` update counts from a sequence

`Vocabulary` properties:

- `cutoff` - same as `unk_cutoff`

- `unk_label` and `counts` can also be accessed as properties
    - `vocab.unk_label`
    - `vocab.counts`

### 2.1. Counts, Vocabulary Membership, and Lookup

Tokens with frequency counts less than the `cutoff` value will be considered not part of the vocabular, even though their entries in the count dictionary are preserved. 

This is useful for changing cut-off without recomputing counts.

In [27]:
# let's define a function to illustrate the relation between counts, membership, and lookup
def test_word(word, vocab):
    # let's lowercase it
    if word != vocab.unk_label:
        word = word.lower() 
    # membership test
    word_membership = word in vocab
    # accessing count
    word_count = vocab[word]
    # lookup
    word_lookup = vocab.lookup(word)
    
    print("Word: '{}', Count: {}, In Vocab: {}, Mapped to: '{}'".format(
        word, word_count, word_membership, word_lookup))
    

In [28]:
for w in ["<UNK>", "the", "1599", "Trento"]:
    test_word(w, vocab)

Word: '<UNK>', Count: 2, In Vocab: True, Mapped to: '<UNK>'
Word: 'the', Count: 993, In Vocab: True, Mapped to: 'the'
Word: '1599', Count: 1, In Vocab: False, Mapped to: '<UNK>'
Word: 'trento', Count: 0, In Vocab: False, Mapped to: '<UNK>'


### 2.2. Cut-Off and Vocabulary Size
The cut-off value influences not only membership checking but also the result of getting the size of the vocabulary using the built-in `len`. Note that while the number of keys in the vocabulary's counter stays the same, the items in the vocabulary differ depending on the cut-off.

In [29]:
# Let's define another vocabulary with cut-off 1
# Notice that we are passing counts from the original vocabulary
vocab1 = Vocabulary(vocab.counts, unk_cutoff=1)

In [30]:
print("CutOff 2:", len(vocab))
print("CutOff 1:", len(vocab1))

CutOff 2: 1867
CutOff 1: 4717


In [31]:
print("CutOff 2 Counts:", len(vocab.counts))
print("CutOff 1 Counts:", len(vocab1.counts))

CutOff 2 Counts: 4716
CutOff 1 Counts: 4716


#### Exercise
- lookup in vocabulary
    - "trento"
    - "trento is the capital city of trentino"
        - split into a list
- update vocabulary with "trento is the capital city of trentino" (splitting)
    - do the lookup again to see the effect
- create another vocabulary changing the cut-off to `1`
    - do the lookup again to see the effect

## 3. Training Language Model

Having prepared the data as described above, the ngram model training with `NLTK` boils down to counting ngrams as presented above, specifying the highest order ngram size.

In [32]:
# Let's prepare data on hamlet
from nltk.lm.preprocessing import padded_everygram_pipeline
data = [[w.lower() for w in sent] for sent in gutenberg.sents('shakespeare-hamlet.txt')]
padded_ngrams, flat_text = padded_everygram_pipeline(2, data)

In [33]:
# Let's train a `Maximum Likelihood Estimator (MLE)
from nltk.lm import MLE

mle_lm = MLE(2)

Initialization of the Language Model creates:
- empty vocabulary
- empty counts

These are populated once we `fit` the model with training data.

In [34]:
print(mle_lm.vocab)
print(mle_lm.counts)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 0 items>
<NgramCounter with 1 ngram orders and 0 ngrams>


In [35]:
mle_lm.fit(padded_ngrams, flat_text)

In [36]:
print(mle_lm.vocab)
print(mle_lm.counts)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 4719 items>
<NgramCounter with 2 ngram orders and 84038 ngrams>


## 4. Using Ngram Language Models

A statistical [language model](https://en.wikipedia.org/wiki/Language_model) is a probability distribution over sequences of words. Given such a sequence, say of length $n$, it assigns a probability $P(w_{1},\ldots ,w_{n})$ ($P(w_{1}^{n})$, for compactness) to the whole sequence (using Chain Rule). Consequently, the unigram and bigram probabilities computed above constitute an ngram language model of our corpus.

It is more useful for Natural Language Processing to have a __probability__ of a sequence being legal, rather than a grammar's __boolean__ decision whether it is legal or not.

### 4.1. Computing Probability of a Sequence (Scoring)

The most common usage of a language model is to compute probability of a sequence.

#### Probability of a Sequence

Probability of a sequence is computed as a product of conditional probabilities ([Chain Rule](https://en.wikipedia.org/wiki/Chain_rule_(probability))). 

$$P(w_{1}^{n}) = P(w_1) P(w_2|w_1) P(w_3|w_1^2) ... P(w_n|w_{1}^{n-1}) = \prod_{i=1}^{n}{P(w_i|w_{1}^{i-1})}$$

The order of ngram makes a simplifying assumption that probability of a current word only depends on previous $N - 1$ elements. Thus, it truncates previous context (history) to length $N - 1$.

$$P(w_i|w_{1}^{i-1}) \approx P(w_i|w_{i-N+1}^{i-1})$$

Consequently we have:

N-gram   | Equation                   |
:--------|:---------------------------|
unigram  | $$P(w_i)$$                 |
bigram   | $$P(w_i|w_{i-1})$$         |
trigram  | $$P(w_i|w_{i-2},w_{i-1})$$ |

The probability of the whole sequence applying an ngram model becomes:

$$P(w_{1}^{n}) = \prod_{i=1}^{n}{P(w_i|w_{i-N+1}^{i-1})}$$

#### Calculating Probability from Frequencies

Probabilities of ngrams can be computed *normalizing* frequency counts (*Maximum Likelihood Estimation*): dividing the frequency of an ngram sequence by the frequency of its prefix (*relative frequency*).

N-gram   | Equation                      
:--------|:------------------------------
Unigram  | $$p(w_i) = \frac{c(w_i)}{N}$$ 
Bigram   | $$p(w_i | w_{i-1}) = \frac{c(w_{i-1}, w_i)}{c(w_{i-1})}$$ 
Ngram    | $$p(w_i | w_{i-n+1}^{i-1}) = \frac{c(w_{i-n+1}^{i-1}, w_i)}{c(w_{i-n+1}^{i-1})}$$ 

where:
- $N$ is the total number of words in a corpus
- $c(x)$ is the count of occurrences of $x$ in a corpus (x could be unigram, bigram, etc.)

#### 4.1.1. Scoring Methods of NLTK LMs
- `score(word, context=None)` Masks out of vocab (OOV) words (i.e. maps them to `<UNK>`) and computes their model score.
    - scores are **model-specific** (later on this)
    - Parameters:
        - `word (str)` - Word for which we want the score
        - `context (tuple(str))` - Context the word is in. If None, computes unigram score.
        
- `logscore(word, context=None)` - Evaluate the log score of this word in this context.

In [37]:
# A little illustration of the methods
mle_lm.score("the")

0.02278986505095015

In [38]:
mle_lm.score("the", ["of"])

0.09672131147540984

In [39]:
mle_lm.logscore("the", ["of"])

-3.3700223830884077

#### Exercise
Implement a function to compute score of a sequence (i.e. Chain Rule)

- arguments:
    - Language Model
    - List of Tokens

- functionality
    - extracts ngrams w.r.t. LM order (`lm.order`)
    - scores each ngram w.r.t. LM (`lm.score` or `lm.logscore`)
        - mind that `score` takes care of OOV by conterting to `<UNK>` already
    - computes the overal score using chain rule
        - mind the difference between `score` and `logscore`

- compute the scores of the sentences below
    - compute padded and unpadded sequence scores

In [40]:
test_sents = ["the king is dead", "the tzar is dead"]

#### 4.1.2. OOV in MLE
In MLE LM we did not take care of OOV. Consequently, those have `0` counts and probabilities.

In [41]:
print(mle_lm.score("<UNK>"))
print(mle_lm.score("<UNK>", ["<UNK>"]))
print(mle_lm.score("<UNK>", ["the"]))

0.0
0
0.0


In [42]:
# same as above: getting lowercaseed sentences and words 
hamlet_sents = [[w.lower() for w in sent] for sent in gutenberg.sents('shakespeare-hamlet.txt')]
hamlet_words = flatten(hamlet_sents)

In [43]:
# computing vocabulary with cutoff
lex = Vocabulary(hamlet_words, unk_cutoff=2)
print(len(lex))

1867


In [44]:
# replacing words with counts below cutoff with '<UNK>'
hamlet_oov_sents = [list(lex.lookup(sent)) for sent in hamlet_sents]
print(hamlet_oov_sents[0])

['[', 'the', 'tragedie', 'of', 'hamlet', 'by', '<UNK>', '<UNK>', '<UNK>', ']']


In [45]:
# extracting ngrams & words again
padded_ngrams_oov, flat_text_oov = padded_everygram_pipeline(2, hamlet_oov_sents)

In [46]:
lm_oov = MLE(2)
lm_oov.fit(padded_ngrams_oov, flat_text_oov)

In [47]:
print(lm_oov.score("<UNK>"))
print(lm_oov.score("<UNK>", ["<UNK>"]))
print(lm_oov.score("<UNK>", ["the"]))

0.06540897824290828
0.06842105263157895
0.24874118831822759


#### 4.1.3. Data Sparsity: Missing Ngrams
However, even though we have counted ngrams on the data set with `<UNK>`, the counts still do not account for all possible ngrams. Thus, some possible sequences will have zero probability.

__We will address this later.__

In [48]:
print(lm_oov.score("queen"))
print(lm_oov.score("<UNK>", ["queen"]))
print(lm_oov.score("queen", ["<UNK>"]))

0.00018360414945377767
0.0
0.0


### 4.2. Generating Sequences

Ngram Model can be used as an automaton to generate probable legal sequences using the algorithm below.

__Algorithm for Bigram LM__

- $w_{i-1} = $ `<s>`;
- *while* $w_i \neq $ `</s>`

    - stochastically get new word w.r.t. $P(w_i|w_{i-1})$

#### 4.2.1. Generation Method of NLTK LM
- `generate(num_words=1, text_seed=None, random_seed=None)` generates words from the model.
    - Parameters
        - `num_words (int)` - How many words to generate. By default 1.
        - `text_seed` - preceding context the generation should be conditioned on.
            - ngram model is restricted in how much preceding context it can take into account based on max ngram size
        - `random_seed` - A random seed. If provided, makes the random sampling part of generation reproducible.

In [49]:
mle_lm.generate(5)

['</s>', 'of', 'kings', 'in', 'the']

In [50]:
mle_lm.generate(5, text_seed=['king'])

['.', '</s>', 'beene', 'most', 'like']

### 4.3. Language Model Evaluation

Language Models are evaluated using [Perplexity](https://en.wikipedia.org/wiki/Perplexity)

- Measures how well model fits test data
- Probability of test data
- Weighted average branching factor in predicting the next word (lower is better).
- Computed as:

$$ PP(W) = \sqrt[N]{\frac{1}{P(w_1,w_2,...,w_N)}} = \sqrt[N]{\frac{1}{\prod_{i=1}^{N}P(w_i|w_{i-N+1})}}$$

Where $N$ is the number of words in test set.

#### 4.3.1. Evaluation Methods of NLTK LM

`NLTK` provides both **perplexity** and **cross-entropy** as evaluation methods

- `entropy(text_ngrams)` calculates cross-entropy of model for given evaluation text.
    - Parameters
        - `text_ngrams (Iterable(tuple(str)))` A sequence of ngram tuples.

- `perplexity(text_ngrams)` calculates the perplexity of the given text.
    - This is _2 ** cross-entropy_ for the text, so the arguments are the same.

**Perplexity** is related to [**Cross-Entropy**](https://en.wikipedia.org/wiki/Cross_entropy) as:

$$PP(p) = 2^{H(p)}$$

#### Exercise
Compute entropy and perplexity of the `MLE` models (with and without OOV handling) on the bigrams of the test sentences below, treating them as a test set.

- extract bigrams (padded and unpadded)


In [51]:
test_sents = ["the king is dead", "the tzar is dead", "the queen is dead"]

### 4.4. Handling Data Sparseness: Smoothing in NLTK

[`Smoothing(vocabulary, counter, **kwargs)`](https://www.nltk.org/api/nltk.lm.smoothing.html) Class is initialized with the following parameters: 

- `vocabulary (nltk.lm.vocab.Vocabulary)` - The Ngram vocabulary object.
- `counter (nltk.lm.counter.NgramCounter)` - The counts of the vocabulary items.


`Ngram Smoothing Interface` implements [Chen & Goodman 1995](https://aclanthology.org/P96-1041.pdf)'s idea that all smoothing algorithms have certain features in common. Consequently, each Smoothing subclass implements two methods:

- `unigram_score(word)` return unigram score
- `alpha_gamma(word, context)` returns alpha and gamma values (varies w.r.t. method)
    - `gamma` is the value added to missing ngram count
    - `alpha` is the value used to scale the lower order probabilities

The following smoothing methods are implemented:
- `WittenBell(vocabulary, counter, **kwargs)` - Witten-Bell smoothing

- `AbsoluteDiscounting(vocabulary, counter, discount=0.75, **kwargs)` - Smoothing with absolute discount
    - takes `discount=0.75` parameter (default 0.75)
    
- `KneserNey(vocabulary, counter, order, discount=0.1, **kwargs)` - Kneser-Ney Smoothing (an extension of smoothing with a discount)
    

### 4.5. NLTK Language Models

All the language models in NLTK share the same interface and share methods for
- evaluation
- generation
- scoring 

**Scoring** in a language model does the following:
- takes care of OOV words (see above)
- computes `unmasked_score`

The way `unmasked_score` is computed from counts is the difference.

Some langauge models use **smoothing** to account for missing ngrams, some do not (e.g. `MLE`)

Available LM Implementations

- `MLE` - providing MLE ngram model scores.

- Additive Smoothing LMs
    - `Laplace` - Laplace (add one) smoothing ($\gamma = 1$)
    - `Lidstone` - same as `Laplace`, but adds $\gamma$

- Back-Off Language Models
    - `StupidBackoff` - uses `alpha` to scale the lower order probabilities for computing missing ngram scores

- Interpolated Language Models
    - `WittenBellInterpolated` - Interpolated version of Witten-Bell smoothing
    - `KneserNeyInterpolated` - Interpolated version of Kneser-Ney smoothing.
    - `AbsoluteDiscountingInterpolated` - Interpolated version of smoothing with absolute discount

## Lab Exercise: Language Model Evaluation

Evaluate different LMs trained on Shakespeare's Hamlet on Macbeth
- Vary LM parameters
    - ngram size `[2,3]`
    - alpha, discount
- Compute
    - entropy
    - preprlexity


In [52]:
macbeth_sents = [[w.lower() for w in sent] for sent in gutenberg.sents('shakespeare-macbeth.txt')]